# 機械学習入門 - ディープラーニングで物体認識を体験

このノートブックでは、AIの基本的な仕組みを理解し、実際に画像認識を体験します。

## 学習目標
- 機械学習とディープラーニングの基本概念を理解する
- 事前学習済みモデル（VGG16）を使って画像認識を体験する
- AIが画像をどのように「見ている」かを可視化する
- YOLOとOpenCVで人物検出を体験する

## 0. 環境設定

In [ ]:
# 必要なライブラリのインストール
!pip install tensorflow pillow matplotlib opencv-python ultralytics numpy

# 日本語フォントのインストール
!apt-get -y install fonts-ipafont-gothic

In [ ]:
# ライブラリのインポート
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import cv2
from google.colab import files
from IPython.display import display, Image as IPImage
import os

# 日本語フォントの設定
plt.rcParams['font.family'] = 'IPAGothic'

print("セットアップ完了！")

## 1. AIの基本概念を理解しよう

### 機械学習とは？
- **従来のプログラミング**: 人間がルールを書く
- **機械学習**: データからパターンを見つける

### ディープラーニングとは？
- 人間の脳を模倣した「ニューラルネットワーク」
- 層を深くすることで複雑なパターンを学習

In [ ]:
# ニューラルネットワークの可視化
def visualize_neural_network():
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    # 入力層
    input_nodes = 4
    for i in range(input_nodes):
        ax.scatter(0, i - input_nodes/2, s=300, c='blue')
        ax.text(-0.5, i - input_nodes/2, f'入力{i+1}', ha='right', va='center')
    
    # 隠れ層
    hidden_nodes = 5
    for i in range(hidden_nodes):
        ax.scatter(2, i - hidden_nodes/2, s=300, c='green')
        # 入力層から隠れ層への接続
        for j in range(input_nodes):
            ax.plot([0, 2], [j - input_nodes/2, i - hidden_nodes/2], 'gray', alpha=0.3)
    
    # 出力層
    output_nodes = 2
    for i in range(output_nodes):
        ax.scatter(4, i - output_nodes/2, s=300, c='red')
        ax.text(4.5, i - output_nodes/2, f'出力{i+1}', ha='left', va='center')
        # 隠れ層から出力層への接続
        for j in range(hidden_nodes):
            ax.plot([2, 4], [j - hidden_nodes/2, i - output_nodes/2], 'gray', alpha=0.3)
    
    ax.text(0, -3, '入力層', ha='center', fontsize=12)
    ax.text(2, -3, '隠れ層', ha='center', fontsize=12)
    ax.text(4, -3, '出力層', ha='center', fontsize=12)
    
    ax.set_xlim(-1, 5)
    ax.set_ylim(-4, 3)
    ax.axis('off')
    ax.set_title('ニューラルネットワークの構造', fontsize=16)
    
    plt.tight_layout()
    plt.show()

visualize_neural_network()

## 2. VGG16で画像認識を体験

VGG16は、ImageNet（1000種類の物体）で学習済みの画像認識モデルです。

In [ ]:
# VGG16モデルの読み込み
print("VGG16モデルを読み込んでいます...")
model = VGG16(weights='imagenet')
print("モデルの読み込み完了！")
print(f"モデルの層数: {len(model.layers)}層")

In [ ]:
# 画像認識関数
def predict_image(image_path):
    """
    画像を読み込んで物体認識を行う
    """
    # 画像の読み込みと前処理
    img = Image.open(image_path)
    img = img.resize((224, 224))  # VGG16の入力サイズ
    
    # numpy配列に変換
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # 予測
    predictions = model.predict(img_array)
    decoded = decode_predictions(predictions, top=5)[0]
    
    return img, decoded

def display_predictions(image_path):
    """
    予測結果を表示
    """
    img, predictions = predict_image(image_path)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # 画像表示
    ax1.imshow(img)
    ax1.axis('off')
    ax1.set_title('入力画像', fontsize=14)
    
    # 予測結果のグラフ
    labels = [p[1] for p in predictions]
    scores = [p[2] for p in predictions]
    
    bars = ax2.barh(labels, scores)
    ax2.set_xlabel('確率', fontsize=12)
    ax2.set_title('認識結果 Top 5', fontsize=14)
    ax2.set_xlim(0, 1)
    
    # スコアを表示
    for bar, score in zip(bars, scores):
        ax2.text(score + 0.01, bar.get_y() + bar.get_height()/2,
                f'{score:.1%}', va='center')
    
    plt.tight_layout()
    plt.show()
    
    print("\n詳細な認識結果:")
    for i, (_, label, score) in enumerate(predictions):
        print(f"{i+1}. {label}: {score:.2%}")

### 画像をアップロードして認識

In [ ]:
# 画像のアップロード
print("認識したい画像をアップロードしてください")
uploaded = files.upload()

# アップロードされた画像を認識
for filename in uploaded.keys():
    print(f"\n=== {filename} の認識結果 ===")
    display_predictions(filename)

## 3. AIが見ている特徴を可視化

CNNの中間層の出力を可視化して、AIがどのような特徴を捉えているか確認します。

In [ ]:
def visualize_feature_maps(image_path, layer_name='block1_conv1'):
    """
    指定した層の特徴マップを可視化
    """
    # 画像の前処理
    img = Image.open(image_path)
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # 特定の層の出力を取得するモデルを作成
    layer_output = model.get_layer(layer_name).output
    feature_model = tf.keras.Model(inputs=model.input, outputs=layer_output)
    
    # 特徴マップを取得
    features = feature_model.predict(img_array)
    
    # 可視化
    n_features = min(features.shape[-1], 16)  # 最大16個の特徴を表示
    fig, axes = plt.subplots(4, 4, figsize=(10, 10))
    
    for i in range(n_features):
        ax = axes[i // 4, i % 4]
        ax.imshow(features[0, :, :, i], cmap='viridis')
        ax.axis('off')
        ax.set_title(f'特徴 {i+1}')
    
    plt.suptitle(f'{layer_name}層の特徴マップ', fontsize=16)
    plt.tight_layout()
    plt.show()

# 最初にアップロードされた画像で特徴を可視化
if uploaded:
    first_image = list(uploaded.keys())[0]
    print("初期層の特徴マップ:")
    visualize_feature_maps(first_image, 'block1_conv1')
    
    print("\n中間層の特徴マップ:")
    visualize_feature_maps(first_image, 'block3_conv1')

## 4. 人物検出の体験

YOLOとOpenCVを使って人物検出を行います。

In [ ]:
# YOLOのセットアップ
try:
    from ultralytics import YOLO
    print("YOLOv8をダウンロードしています...")
    yolo_model = YOLO('yolov8n.pt')  # nanoモデル（軽量）
    USE_YOLO = True
    print("YOLO準備完了！")
except Exception as e:
    print(f"YOLOの読み込みに失敗しました: {e}")
    print("OpenCVのみを使用します")
    USE_YOLO = False

In [ ]:
def detect_with_yolo(image_path):
    """
    YOLOで人物検出
    """
    results = yolo_model(image_path)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    person_count = 0
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # クラス0が人物
            if box.cls == 0:
                person_count += 1
                x1, y1, x2, y2 = box.xyxy[0]
                cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(img, f'Person {box.conf[0]:.2f}', 
                           (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 
                           0.5, (0, 255, 0), 2)
    
    return img, person_count

def detect_with_opencv(image_path):
    """
    OpenCV HOGで人物検出
    """
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
    
    (rects, weights) = hog.detectMultiScale(img, 
                                           winStride=(4, 4),
                                           padding=(8, 8),
                                           scale=1.05)
    
    for (x, y, w, h) in rects:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    return img, len(rects)

def compare_detection_methods(image_path):
    """
    検出手法の比較
    """
    fig, axes = plt.subplots(1, 3 if USE_YOLO else 2, figsize=(15, 5))
    
    # 元画像
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    axes[0].imshow(img)
    axes[0].set_title('元画像')
    axes[0].axis('off')
    
    # YOLO検出
    if USE_YOLO:
        yolo_img, yolo_count = detect_with_yolo(image_path)
        axes[1].imshow(yolo_img)
        axes[1].set_title(f'YOLO検出: {yolo_count}人')
        axes[1].axis('off')
    
    # OpenCV検出
    opencv_img, opencv_count = detect_with_opencv(image_path)
    idx = 2 if USE_YOLO else 1
    axes[idx].imshow(opencv_img)
    axes[idx].set_title(f'OpenCV HOG検出: {opencv_count}人')
    axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 人物が写っている画像をアップロード
print("人物が写っている画像をアップロードしてください")
uploaded_person = files.upload()

# 人物検出を実行
for filename in uploaded_person.keys():
    print(f"\n=== {filename} の人物検出結果 ===")
    compare_detection_methods(filename)

## 5. 工業製品での実験

工業製品の画像で認識を試してみましょう。

In [ ]:
# 工業製品の画像をアップロード
print("工業製品（ギア、ネジ、回路基板など）の画像をアップロードしてください")
uploaded_industrial = files.upload()

# 認識を実行
for filename in uploaded_industrial.keys():
    print(f"\n=== {filename} の認識結果 ===")
    display_predictions(filename)
    
print("\n💡 考察：")
print("一般的なAIモデルは工業製品の詳細な分類は苦手です。")
print("だからこそ、Teachable Machineで専用のモデルを作る必要があります！")

## まとめ

このノートブックで学んだこと：
1. **機械学習の基本概念** - パターンを見つける技術
2. **画像認識の体験** - VGG16で1000種類の物体認識
3. **特徴の可視化** - AIが注目している部分を確認
4. **人物検出** - YOLOとOpenCVの比較
5. **専用AIの必要性** - 工業製品には専門的なモデルが必要

### 次のステップ
- Teachable Machineで自分専用のAIモデルを作成
- 傷検出AIの開発に挑戦

In [ ]:
# 作成した画像を保存
save_dir = 'ml_intro_results'
os.makedirs(save_dir, exist_ok=True)
print(f"結果は '{save_dir}' フォルダに保存されています")

# Google Driveに保存する場合
from google.colab import drive
drive.mount('/content/drive')
# !cp -r ml_intro_results /content/drive/MyDrive/